In [8]:
import tweepy as tw
from tweepy import *
import requests
import os
import numpy as np
import pandas as pd
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential
import json
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pycountry
import re
import string
from textblob import TextBlob
import sys
import matplotlib.pyplot as plt

from dotenv import load_dotenv
load_dotenv()

True

In [9]:
# your Twitter API key and API secret & Microsoft Azure API Credentials
my_api_key = os.getenv("twitter_api_key")
my_api_secret = os.getenv("twitter_secret_api_key")
language_api_key = os.getenv("azure_language")
endpoint = os.getenv("endpoint")
ta_credential = AzureKeyCredential(language_api_key)
text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint, credential=ta_credential)


#authenticate
auth = tw.OAuthHandler(my_api_key, my_api_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [10]:
search_query = "#ETH -filter:retweets"

In [11]:
# get tweets from the API
tweets = tw.Cursor(api.search_tweets,
              q=search_query,
              lang="en",
              since="2021-09-16").items(10)

# store the API responses in a list
tweets_copy = []
for tweet in tweets:
    tweets_copy.append(tweet)
    
print("Total Tweets fetched:", len(tweets_copy))

Unexpected parameter: since


Total Tweets fetched: 10


In [12]:
# intialize the dataframe
tweets_df = pd.DataFrame()

# populate the dataframe
for tweet in tweets_copy:
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
        text = api.get_status(id=tweet.id, tweet_mode='extended').full_text
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source}))
    tweets_df = tweets_df.reset_index(drop=True)

# show the dataframe
tweets_df.head(5)

,user_name,user_location,user_description,user_verified,date,text,hashtags,source
0,Raghu Domainer,,https://t.co/xzXTGMYn7x l https://t.co/s25tFJc...,False,2021-12-10 02:08:23+00:00,https://t.co/t5slIj39bR domain name was recent...,None,Twitter Web App
1,Olusegun Rebecca,,,False,2021-12-10 02:08:18+00:00,@AirdropStario This is definitely going to be ...,None,Twitter for Android
2,OK👌🏼,United States,I offer some free recommendations and some are...,False,2021-12-10 02:08:16+00:00,Binance Will List Flux (FLUX)\nhttps://t.co/u5...,"[Solana, BNB, ADA, shiba, SHIB, SHIBAINU, SHIB...",Twitter for Android
3,Daman 💨,,https://t.co/cctGuCncJV,False,2021-12-10 02:08:05+00:00,@NFTCompanyy Check out this account on OpenSea...,"[NFTs, NFTCommunity, NFTGiveaway, NFTcollectio...",Twitter for Android
4,Daman 💨,,https://t.co/cctGuCncJV,False,2021-12-10 02:07:59+00:00,@LionNfts Check out this account on OpenSea\n\...,"[NFTs, NFTCommunity, NFTGiveaway, NFTcollectio...",Twitter for Android


In [ ]:
df=str(tweets_df.iloc[:,5])

In [ ]:
Microsoft Azure Cognitive Text Analytics 

documents = [df]

response = text_analytics_client.analyze_sentiment(documents, language="en")
result = [doc for doc in response if not doc.is_error]

for doc in result:
    print("Overall sentiment: {}".format(doc.sentiment))
    print("Scores: positive={}; neutral={}; negative={} \n".format(
        doc.confidence_scores.positive,
        doc.confidence_scores.neutral,
        doc.confidence_scores.negative,
    ))
print(search_query)
